# I. Tiến Hành thu thập dữ liệu

---

#### Import thư viện

In [22]:
import requests as r
import csv
import pandas as pd
from bs4 import BeautifulSoup
from datetime import date

#### Dùng requests để lấy data từ trang ```https://www.worldometers.info/coronavirus```

In [23]:
headers = {'User-Agent': 'Mozilla/5.0'}
page = r.get("https://www.worldometers.info/coronavirus", headers)
soup  = BeautifulSoup(page.text, 'html.parser')

#### Hàm lấy tên của cột

In [24]:
def getTitle():
    title = soup.find_all("thead")
    # title
    result_title = []
    for t in title:
        result = t.find_all("th")
        for r in result[1:]:
            temp = r.text
            temp = temp.replace("\n", "")
            result_title.append(temp)
        break


    return result_title

#### Hàm lấy dữ liệu từ now, yesterday, 2 days ago

In [25]:
def getData(_type):
    result = []
    # now
    if (_type == 1):
        data = soup.find_all("table", {"id": "main_table_countries_today"})
    # yesterday
    elif (_type == 2):
        data = soup.find_all("table", {"id": "main_table_countries_yesterday"})
    # 2 days ago
    else:
        data = soup.find_all("table", {"id": "main_table_countries_yesterday2"})
    
    for D in data:
        data = D.find_all("tbody")
        for d in data:
            d.find_all("tr")
            for value in d:
                try:
                    temp = value["style"]
                    if (temp != "display: none"):
                        result.append(value)
                except:
                    None
            break    
        break
        
    return result
    

#### Thực hiện các thao tác lấy dữ liệu 

In [26]:
# lay tieu de cua cot
result_title = getTitle()

# # lấy thông tin của ngày hôm nay
# result_now = getData(1)
# # lấy thông tin của 2 ngày trước
# result_twoDaysAgo = getData(3)

# lấy thông tin của ngày hôm qua
result_yesterday = getData(2)



#### Khi chạy trên khối này, dữ liệu sẽ được crawl và đưa vào file csv 
   + Lựa chọn 3 loại dữ liệu đưa vào csv ( ```result_twoDaysAgo```, ```result_yesterday```, ```result_now``` )
   + Thêm ngày để bổ sung vào thuộc tính Date

In [27]:
with open("Data\Covid.csv", "a", encoding="UTF8") as f_object:
    writer_object = csv.writer(f_object)
    
    # write title to covid.csv
    writer_object.writerow(result_title)
    # write data
    for r in result_yesterday:
        temp = []
        for i in r.find_all("td")[1:]:
            temp.append(i.text)
        
        #write
        writer_object.writerow(temp)
    f_object.close()

In [28]:
data_covid = pd.read_csv("Data\Covid.csv")

In [29]:
data_covid

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,...,TotalTests,Tests/1M pop,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl,New Cases/1M pop,New Deaths/1M pop,Active Cases/1M pop
0,China,"115,466","+1,938","4,636",NaN,"103,600",+169,"7,230",6,80,...,"160,000,000","111,163","1,439,323,776",Asia,"12,465","310,467",9.0,1,NaN,5
1,USA,"81,174,677","+5,143","993,811",118.0,"56,071,103","+155,965","24,109,763","4,250","242,828",...,"964,386,655","2,884,885","334,289,437",North America,4,336,0.0,15,0.40,"72,122"
2,India,"42,993,494","+2,503","515,907",57.0,"42,441,449","+4,377","36,138","8,944","30,644",...,"778,520,151","554,906","1,402,976,534",Asia,33,"2,719",2.0,2,0.04,26
3,Brazil,"29,368,776","+18,397","655,139",146.0,"27,742,324","+70,731","971,313","8,318","136,524",...,"63,776,166","296,471","215,117,939",South America,7,328,3.0,86,0.70,"4,515"
4,France,"23,514,144","+60,422","140,109",29.0,"22,163,944","+17,144","1,210,091","1,928","358,895",...,"246,629,975","3,764,303","65,518,106",Europe,3,468,0.0,922,0.40,"18,470"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,MS Zaandam,9,NaN,2,NaN,7,NaN,0,NaN,NaN,...,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
223,Marshall Islands,7,NaN,,NaN,7,NaN,0,NaN,117,...,NaN,NaN,"59,866",Australia/Oceania,"8,552",NaN,NaN,NaN,NaN,NaN
224,Saint Helena,2,NaN,,NaN,2,NaN,0,NaN,327,...,NaN,NaN,"6,108",Africa,"3,054",NaN,NaN,NaN,NaN,NaN
225,Micronesia,1,NaN,,NaN,1,NaN,0,NaN,9,...,NaN,NaN,"117,064",Australia/Oceania,"117,064",NaN,NaN,NaN,NaN,NaN
